<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ENEE439d-TEXTML/TextML/blob/master/input.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ENEE439d-TEXTML/TextML/blob/master/input.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Setup

In [ ]:
!pip install -q -U "tensorflow-text==2.8.*" # A dependency of the preprocessing for BERT inputs
!pip install -q tf-models-official==2.7.0 # For adamW
!pip install focal-loss # focal loss implmnetion for tf

     |████████████████████████████████| 4.9 MB 5.3 MB/s 
     |████████████████████████████████| 462 kB 46.0 MB/s 
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 47.8 MB 3.4 MB/s 
     |████████████████████████████████| 237 kB 46.6 MB/s 
     |████████████████████████████████| 99 kB 8.7 MB/s 
     |████████████████████████████████| 90 kB 8.3 MB/s 
     |████████████████████████████████| 1.2 MB 40.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 596 kB 43.8 MB/s 
     |████████████████████████████████| 352 kB 43.9 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 


In [ ]:
import pandas as pd #basic imports
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras import layers
import re

In [ ]:
from sklearn.model_selection import train_test_split # https://www.tensorflow.org/text/tutorials/classify_text_with_bert
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization, bert  # to create AdamW optimizer
from focal_loss import SparseCategoricalFocalLoss
import official.nlp.bert.tokenization

# Model Without Context

## Import Data

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read in data from google drive
filepath = '/content/drive/MyDrive/801_paper_dataset.csv' #'data.csv'
df1= pd.read_csv(filepath)

In [ ]:
df1

## Remove Citations and Useless Characters

In [ ]:
# create regex to remove citations and useles characters

context=df1['text']

re1= "\(((([A-Za-z]+ *)+(, \d+))+(; )*)+\)" # matches author and author, year
re_year=",? \(?\d{4}\)?" # match , {4 digits} which may be wrapped in () 
re_and="(and|&) "
re_auth="((\w+, )*(\w+ )+)"
re_et= re_auth+"et al\. ?"+re_year # matches author et al. , year
re_2a= re_auth+"("+re_and+"((\w+ *))?)?"+re_year # matches author and author, year
re_sep="((; )|( "+re_and+"))*"# match the '; ' gap or ' and ' gap
re_para_year="\(\d{4}\)"
re_in_brack="\[*\]"
re_apa =re_in_brack+"|"+re_para_year+"|"+"\(?("+"(\(?"+re_2a+"|"+re_et+"\)?)"+re_sep+")+"
print(re_apa)

\[*\]|\(\d{4}\)|\(?((\(?((\w+, )*(\w+ )+)((and|&) ((\w+ *))?)?,? \(?\d{4}\)?|((\w+, )*(\w+ )+)et al\. ?,? \(?\d{4}\)?\)?)((; )|( (and|&) ))*)+


In [ ]:
# remove citations and useless characters

def remove_citation(text,regex=re_apa):
  text1=text
  rem_len=0
  pattern= re.compile(regex)
  while True:
    matches=pattern.search(text1)
    #print(matches)
    if matches == None:
      break

    spn=matches.span()
    text1=text1[0:spn[0]]+text1[spn[1]:-1]
    cit_len=spn[1]-spn[0]
    rem_len+=cit_len
  
  if len(text) >0:
    percent_removed=rem_len/len(text)
  else:
    percent_removed=1
  return text1,percent_removed 

print("example of preprocessed data: \n")
print(context[5])
remove_citation(context[5],regex=re_apa)

example of preprocessed data: 

For instance, in the aforementioned GreenOrbs project, the mobile user needs to collect data from the entire network in many scenarios, such as the periodical measurement of concentration of carbon dioxide for the scientific analysis, the network abnormal event detection, etc. We claim that the ubiquitous data collection problem studied in this paper is essentially different from conventional mobile sink-based data collection [18], [19].


('For instance, in the aforementioned GreenOrbs project, the mobile user needs to collect data from the entire network in many scenarios, such as the periodical measurement of concentration of carbon dioxide for the scientific analysis, the network abnormal event detection, etc. We claim that the ubiquitous data collection problem studied in this paper is essentially different from conventional mobile sink-based data collection [18, [19',
 0.0045351473922902496)

In [ ]:
# clean dataset

output=df1['text'].apply(lambda x: remove_citation(text=x,regex=re_apa)) #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
df_o = pd.DataFrame(list(output), columns =['clean','p_rem'])
output_1=df_o['clean'].apply(lambda x: remove_citation(text=x,regex='[^\w_0-9 ]+')) 
df_o_1 = pd.DataFrame(list(output_1), columns =['clean','p_rem'])
#df_o.head()

df1['text_clean']=df_o_1['clean']
df1['text_clean_len']=df_o_1['clean'].apply(len)
df1['p_rem']=df_o['p_rem']
df1

,Unnamed: 0,paper_number,line number,text,text_clean,text_clean_len,p_rem
0,0,40000,12.0,With the data collected from the entire networ...,With the data collected from the entire networ...,279,0.023739
1,1,40000,16.0,"In the GreenOrbs project [9], more than 300 se...",In the GreenOrbs project 9 more than 300 senso...,192,0.004902
2,2,40000,21.0,The data collection tree is fixed and suffices...,The data collection tree is fixed and suffices...,105,0.038760
3,3,40000,25.0,"Unveiled by Kusy et al. [16], building the dat...",Unveiled by Kusy et al 16 building the data co...,109,0.008547
4,4,40000,28.0,"Most existing works, however, assume that the ...",Most existing works however assume that the mo...,235,0.008097
...,...,...,...,...,...,...,...
19820,19820,40800,62.0,Each point in \mathcal {A} corresponds to an e...,Each point in mathcal A corresponds to an equi...,203,0.004695
19821,19821,40800,71.0,"Definitions for {\bf G}, {\bf F}(\cdot) and {\...",Definitions for bf G bf Fcdot and bf Hcdot are...,51,0.012987
19822,19822,40800,74.0,"A configuration (\mathbf {q},\mathbf {u}) is a...",A configuration mathbf qmathbf u is a stable e...,1739,0.001762
19823,19823,40800,87.0,We define an homogeneous transformation matrix...,We define an homogeneous transformation matrix...,7948,0.000894


In [ ]:
data_for_input= [[s] for s in list(df1['text_clean'])]

In [ ]:
data_for_input[0]

['With the data collected from the entire network the sensor network supports a variety of applications including security surveillance 1 2 3 localization 4 information enquiry and transmission 5 6 7 8 etc In this paper we consider the ubiquitous data collection by mobile users in']

## Bert Model

In [ ]:
#@title Choose a BERT model to fine-tune (Taken from tutorial)

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()

In [ ]:
classifier_model.load_weights('/content/drive/MyDrive/Text-ML/checkpoint')

In [ ]:
predictions = classifier_model.predict(data_for_input, verbose=1)

620/620 [==============================] - 1431s 2s/step


In [ ]:
predictions

array([[1.09003182e-03, 4.61928576e-01, 5.36981463e-01],
       [1.57016341e-03, 7.29224443e-01, 2.69205421e-01],
       [5.42949326e-03, 2.77287245e-01, 7.17283309e-01],
       ...,
       [2.60356665e-01, 4.73729193e-01, 2.65914142e-01],
       [6.79522101e-03, 8.72569919e-01, 1.20634854e-01],
       [2.77002866e-04, 9.66069639e-01, 3.36533822e-02]], dtype=float32)

## Results

In [ ]:
#take max of the softmax output and map values back to (-1,0,1)
my_predictions_final = np.argmax(predictions, axis=1) - 1

final_df = df1.copy()
final_df['sentiment predictions'] = my_predictions_final
final_df

,Unnamed: 0,paper_number,line number,text,text_clean,text_clean_len,p_rem,sentiment predictions
0,0,40000,12.0,With the data collected from the entire networ...,With the data collected from the entire networ...,279,0.023739,1
1,1,40000,16.0,"In the GreenOrbs project [9], more than 300 se...",In the GreenOrbs project 9 more than 300 senso...,192,0.004902,0
2,2,40000,21.0,The data collection tree is fixed and suffices...,The data collection tree is fixed and suffices...,105,0.038760,1
3,3,40000,25.0,"Unveiled by Kusy et al. [16], building the dat...",Unveiled by Kusy et al 16 building the data co...,109,0.008547,1
4,4,40000,28.0,"Most existing works, however, assume that the ...",Most existing works however assume that the mo...,235,0.008097,0
...,...,...,...,...,...,...,...,...
19820,19820,40800,62.0,Each point in \mathcal {A} corresponds to an e...,Each point in mathcal A corresponds to an equi...,203,0.004695,0
19821,19821,40800,71.0,"Definitions for {\bf G}, {\bf F}(\cdot) and {\...",Definitions for bf G bf Fcdot and bf Hcdot are...,51,0.012987,0
19822,19822,40800,74.0,"A configuration (\mathbf {q},\mathbf {u}) is a...",A configuration mathbf qmathbf u is a stable e...,1739,0.001762,0
19823,19823,40800,87.0,We define an homogeneous transformation matrix...,We define an homogeneous transformation matrix...,7948,0.000894,0


In [ ]:
#check to see if any are predicted with negative sentiment
final_df.loc[final_df['sentiment predictions'] == -1]

,Unnamed: 0,paper_number,line number,text,text_clean,text_clean_len,p_rem,sentiment predictions
25,25,40000,369.0,The mobile user probing process does not intro...,The mobile user probing process does not intro...,154,0.006173,-1
147,147,40003,255.0,The mobile user probing process does not intro...,The mobile user probing process does not intro...,154,0.006173,-1
394,394,40014,12.0,These methods have inherent difficulties and v...,These methods have inherent difficulties and v...,103,0.009346,-1
433,433,40015,73.0,"However, [13] did not provide exact SOP closed...",However 13 did not provide exact SOP closedfor...,82,0.011111,-1
515,515,40019,14.0,OMA schemes can hardly meet the coming require...,OMA schemes can hardly meet the coming require...,127,0.007407,-1
...,...,...,...,...,...,...,...,...
19724,19724,40798,41.0,While theoretically well-motivated learning me...,While theoretically wellmotivated learning met...,214,0.024490,-1
19732,19732,40798,49.0,"However, in practice, interactions between dis...",However in practice interactions between dista...,169,0.005650,-1
19765,19765,40799,18.0,Though some of the same methods can be extende...,Though some of the same methods can be extende...,134,0.007194,-1
19768,19768,40799,22.0,While this is a practical choice to solve real...,While this is a practical choice to solve real...,96,0.009804,-1


# Model With Context

## Import Data

In [44]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# read in data from google drive
filepath = '/content/drive/MyDrive/801_paper_dataset_with_contexts.csv' #'data.csv'
df1= pd.read_csv(filepath)

In [51]:
df1['text'] = df1['text (with context)']
df1

,Unnamed: 0,paper_number,line number,text (with context),text
0,0,40000,12.0,Pastseveral years have witnessed a great succe...,Pastseveral years have witnessed a great succe...
1,1,40000,16.0,Such a pervasive usage of sensor networks expl...,Such a pervasive usage of sensor networks expl...
2,2,40000,21.0,The ubiquitous data collection problem conside...,The ubiquitous data collection problem conside...
3,3,40000,25.0,To efficiently deliver network-wide data to th...,To efficiently deliver network-wide data to th...
4,4,40000,28.0,"Besides, the routing transitions between diffe...","Besides, the routing transitions between diffe..."
...,...,...,...,...,...
19820,19820,40800,62.0,We define the set \mathcal {A} \subset \mathbb...,We define the set \mathcal {A} \subset \mathbb...
19821,19821,40800,71.0,"The map \Phi is injective, i.e. for each (\mat...","The map \Phi is injective, i.e. for each (\mat..."
19822,19822,40800,74.0,"Here also, the matrices {\bf M} and {\bf J} ar...","Here also, the matrices {\bf M} and {\bf J} ar..."
19823,19823,40800,87.0,Perturbation Mapping From \mathcal {A} to \mat...,Perturbation Mapping From \mathcal {A} to \mat...


## Remove Citations and Useless Characters

In [52]:
# create regex to remove citations and useles characters

context=df1['text']

re1= "\(((([A-Za-z]+ *)+(, \d+))+(; )*)+\)" # matches author and author, year
re_year=",? \(?\d{4}\)?" # match , {4 digits} which may be wrapped in () 
re_and="(and|&) "
re_auth="((\w+, )*(\w+ )+)"
re_et= re_auth+"et al\. ?"+re_year # matches author et al. , year
re_2a= re_auth+"("+re_and+"((\w+ *))?)?"+re_year # matches author and author, year
re_sep="((; )|( "+re_and+"))*"# match the '; ' gap or ' and ' gap
re_para_year="\(\d{4}\)"
re_in_brack="\[*\]"
re_apa =re_in_brack+"|"+re_para_year+"|"+"\(?("+"(\(?"+re_2a+"|"+re_et+"\)?)"+re_sep+")+"
print(re_apa)

\[*\]|\(\d{4}\)|\(?((\(?((\w+, )*(\w+ )+)((and|&) ((\w+ *))?)?,? \(?\d{4}\)?|((\w+, )*(\w+ )+)et al\. ?,? \(?\d{4}\)?\)?)((; )|( (and|&) ))*)+


In [53]:
# remove citations and useless characters

def remove_citation(text,regex=re_apa):
  text1=text
  rem_len=0
  pattern= re.compile(regex)
  while True:
    matches=pattern.search(text1)
    #print(matches)
    if matches == None:
      break

    spn=matches.span()
    text1=text1[0:spn[0]]+text1[spn[1]:-1]
    cit_len=spn[1]-spn[0]
    rem_len+=cit_len
  
  if len(text) >0:
    percent_removed=rem_len/len(text)
  else:
    percent_removed=1
  return text1,percent_removed 

print("example of preprocessed data: \n")
print(context[5])
remove_citation(context[5],regex=re_apa)

example of preprocessed data: 

The mobile user uses a handheld device to communicate with sensor nodes in the network.The mobile user roams within the network and instantly accesses network-wide data on a need basis.For instance, in the aforementioned GreenOrbs project, the mobile user needs to collect data from the entire network in many scenarios, such as the periodical measurement of concentration of carbon dioxide for the scientific analysis, the network abnormal event detection, etc. We claim that the ubiquitous data collection problem studied in this paper is essentially different from conventional mobile sink-based data collection [18], [19].In previous works, the sink mobility has been introduced to benefit the data collection operation, e.g., to improve the network lifetime. The mobile sink is essentially cooperative to the data collection.


('The mobile user uses a handheld device to communicate with sensor nodes in the network.The mobile user roams within the network and instantly accesses network-wide data on a need basis.For instance, in the aforementioned GreenOrbs project, the mobile user needs to collect data from the entire network in many scenarios, such as the periodical measurement of concentration of carbon dioxide for the scientific analysis, the network abnormal event detection, etc. We claim that the ubiquitous data collection problem studied in this paper is essentially different from conventional mobile sink-based data collection [18, [19.In previous works, the sink mobility has been introduced to benefit the data collection operation, e.g., to improve the network lifetime. The mobile sink is essentially cooperative to the data collectio',
 0.0024096385542168677)

In [54]:
# clean dataset

output=df1['text'].apply(lambda x: remove_citation(text=x,regex=re_apa)) #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
df_o = pd.DataFrame(list(output), columns =['clean','p_rem'])
output_1=df_o['clean'].apply(lambda x: remove_citation(text=x,regex='[^\w_0-9 ]+')) 
df_o_1 = pd.DataFrame(list(output_1), columns =['clean','p_rem'])
#df_o.head()

df1['text_clean']=df_o_1['clean']
df1['text_clean_len']=df_o_1['clean'].apply(len)
df1['p_rem']=df_o['p_rem']
df1

,Unnamed: 0,paper_number,line number,text (with context),text,text_clean,text_clean_len,p_rem
0,0,40000,12.0,Pastseveral years have witnessed a great succe...,Pastseveral years have witnessed a great succe...,Pastseveral years have witnessed a great succe...,616,0.011713
1,1,40000,16.0,Such a pervasive usage of sensor networks expl...,Such a pervasive usage of sensor networks expl...,Such a pervasive usage of sensor networks expl...,849,0.001130
2,2,40000,21.0,The ubiquitous data collection problem conside...,The ubiquitous data collection problem conside...,The ubiquitous data collection problem conside...,523,0.010508
3,3,40000,25.0,To efficiently deliver network-wide data to th...,To efficiently deliver network-wide data to th...,To efficiently deliver networkwide data to the...,701,0.001379
4,4,40000,28.0,"Besides, the routing transitions between diffe...","Besides, the routing transitions between diffe...",Besides the routing transitions between differ...,794,0.002421
...,...,...,...,...,...,...,...,...
19820,19820,40800,62.0,We define the set \mathcal {A} \subset \mathbb...,We define the set \mathcal {A} \subset \mathbb...,We define the set mathcal A subset mathbb R6 b...,846,0.001629
19821,19821,40800,71.0,"The map \Phi is injective, i.e. for each (\mat...","The map \Phi is injective, i.e. for each (\mat...",The map Phi is injective ie for each mathbf qm...,498,0.002695
19822,19822,40800,74.0,"Here also, the matrices {\bf M} and {\bf J} ar...","Here also, the matrices {\bf M} and {\bf J} ar...",Here also the matrices bf M and bf J are compl...,2005,0.001545
19823,19823,40800,87.0,Perturbation Mapping From \mathcal {A} to \mat...,Perturbation Mapping From \mathcal {A} to \mat...,Perturbation Mapping From mathcal A to mathcal...,8262,0.000860


In [55]:
data_for_input= [[s] for s in list(df1['text_clean'])]

In [56]:
data_for_input[0]

['Pastseveral years have witnessed a great success of Wireless Sensor Networks WSNsRecent advances in Wireless Sensor Network technologies provide people the ability to better understand the physical worldWith the data collected from the entire network the sensor network supports a variety of applications including security surveillance 1 2 3 localization 4 information enquiry and transmission 5 6 7 8 etc In this paper we consider the ubiquitous data collection by mobile users in the wireless sensor networkMobile users are equipped with handheld devices that communicate with sensor nodes and instantly access th']

## Bert Model

In [57]:
#@title Choose a BERT model to fine-tune (Taken from tutorial)

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [58]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [59]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [60]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [61]:
classifier_model = build_classifier_model()

In [62]:
classifier_model.load_weights('/content/drive/MyDrive/Text-ML/checkpoint')

In [63]:
predictions = classifier_model.predict(data_for_input, verbose=1)

620/620 [==============================] - 161s 259ms/step


In [64]:
predictions

array([[0.00252013, 0.03179268, 0.9656872 ],
       [0.00867456, 0.1951602 , 0.7961652 ],
       [0.18837187, 0.16807784, 0.6435503 ],
       ...,
       [0.00920738, 0.8955549 , 0.09523776],
       [0.00246297, 0.9683364 , 0.02920074],
       [0.01099892, 0.61643404, 0.37256712]], dtype=float32)

## Results

In [67]:
#take max of the softmax output and map values back to (-1,0,1)
my_predictions_final = np.argmax(predictions, axis=1) - 1

final_context_df = df1.copy()
final_context_df['context_sentiment_predictions'] = my_predictions_final
final_context_df

,Unnamed: 0,paper_number,line number,text (with context),text,text_clean,text_clean_len,p_rem,context_sentiment_predictions
0,0,40000,12.0,Pastseveral years have witnessed a great succe...,Pastseveral years have witnessed a great succe...,Pastseveral years have witnessed a great succe...,616,0.011713,1
1,1,40000,16.0,Such a pervasive usage of sensor networks expl...,Such a pervasive usage of sensor networks expl...,Such a pervasive usage of sensor networks expl...,849,0.001130,1
2,2,40000,21.0,The ubiquitous data collection problem conside...,The ubiquitous data collection problem conside...,The ubiquitous data collection problem conside...,523,0.010508,1
3,3,40000,25.0,To efficiently deliver network-wide data to th...,To efficiently deliver network-wide data to th...,To efficiently deliver networkwide data to the...,701,0.001379,1
4,4,40000,28.0,"Besides, the routing transitions between diffe...","Besides, the routing transitions between diffe...",Besides the routing transitions between differ...,794,0.002421,1
...,...,...,...,...,...,...,...,...,...
19820,19820,40800,62.0,We define the set \mathcal {A} \subset \mathbb...,We define the set \mathcal {A} \subset \mathbb...,We define the set mathcal A subset mathbb R6 b...,846,0.001629,1
19821,19821,40800,71.0,"The map \Phi is injective, i.e. for each (\mat...","The map \Phi is injective, i.e. for each (\mat...",The map Phi is injective ie for each mathbf qm...,498,0.002695,0
19822,19822,40800,74.0,"Here also, the matrices {\bf M} and {\bf J} ar...","Here also, the matrices {\bf M} and {\bf J} ar...",Here also the matrices bf M and bf J are compl...,2005,0.001545,0
19823,19823,40800,87.0,Perturbation Mapping From \mathcal {A} to \mat...,Perturbation Mapping From \mathcal {A} to \mat...,Perturbation Mapping From mathcal A to mathcal...,8262,0.000860,0


In [68]:
#check to see if any are predicted with negative sentiment
final_context_df.loc[final_context_df['context_sentiment_predictions'] == -1]

,Unnamed: 0,paper_number,line number,text (with context),text,text_clean,text_clean_len,p_rem,context_sentiment_predictions
23,23,40000,366.0,Tan et al. [19] further jointly optimizes data...,Tan et al. [19] further jointly optimizes data...,Tan et al 19 further jointly optimizes data ro...,589,0.009449,-1
24,24,40000,368.0,"Moreover, on the application level, Gao et al....","Moreover, on the application level, Gao et al....",Moreover on the application level Gao et al 27...,504,0.007407,-1
25,25,40000,369.0,"On the other hand, some recent works do not as...","On the other hand, some recent works do not as...",On the other hand some recent works do not ass...,438,0.006438,-1
90,90,40001,181.0,Fig. 9. Scaling of the number and type of func...,Fig. 9. Scaling of the number and type of func...,Fig 9 Scaling of the number and type of functi...,794,0.002375,-1
145,145,40003,252.0,Tan et al. [19] further jointly optimizes data...,Tan et al. [19] further jointly optimizes data...,Tan et al 19 further jointly optimizes data ro...,589,0.009449,-1
...,...,...,...,...,...,...,...,...,...
19726,19726,40798,43.0,While theoretically well-motivated learning me...,While theoretically well-motivated learning me...,While theoretically wellmotivated learning met...,484,0.021858,-1
19732,19732,40798,49.0,"This issue has been targeted repeatedly [13], ...","This issue has been targeted repeatedly [13], ...",This issue has been targeted repeatedly 13 34 ...,610,0.012048,-1
19733,19733,40798,50.0,PropNet [13] is tested on 2D ropes and shows a...,PropNet [13] is tested on 2D ropes and shows a...,PropNet 13 is tested on 2D ropes and shows a b...,663,0.009790,-1
19734,19734,40798,51.0,"However, in practice, interactions between dis...","However, in practice, interactions between dis...",However in practice interactions between dista...,771,0.007308,-1


# Context vs No-Context Results

In [69]:
# combine context and non-context dataframea
context_vs_nocontext_df = final_df.copy()
context_vs_nocontext_df['context_text'] = final_context_df['text']
context_vs_nocontext_df['context_text_clean'] = final_context_df['text_clean']
context_vs_nocontext_df['context_text_clean_len'] = final_context_df['text_clean_len']
context_vs_nocontext_df['context_p_rem'] = final_context_df['p_rem']
context_vs_nocontext_df['context_sentiment_predictions'] = final_context_df['context_sentiment_predictions']

In [72]:
# texts where sentiment differs
differences = context_vs_nocontext_df.loc[context_vs_nocontext_df['sentiment predictions'] != context_vs_nocontext_df['context_sentiment_predictions']][['paper_number', 'line number', 'text', 'context_text', 'sentiment predictions', 'context_sentiment_predictions']]

In [83]:
differences

,paper_number,line number,text,context_text,sentiment predictions,context_sentiment_predictions
1,40000,16.0,"In the GreenOrbs project [9], more than 300 se...",Such a pervasive usage of sensor networks expl...,0,1
4,40000,28.0,"Most existing works, however, assume that the ...","Besides, the routing transitions between diffe...",0,1
5,40000,50.0,"For instance, in the aforementioned GreenOrbs ...",The mobile user uses a handheld device to comm...,0,1
6,40000,58.0,Suppose that at sensor node u the mobile user ...,"In this paper, we aim to develop an efficient ...",0,1
12,40000,354.0,"Typical examples include [15], [21].","In addition, according to how does each packet...",1,0
...,...,...,...,...,...,...
19816,40800,45.0,"To the best of the author’s knowledge, this is...","During a collision, the model deviates from th...",0,1
19817,40800,47.0,"In this section, we briefly present the theore...","To the best of the author’s knowledge, this is...",0,1
19818,40800,48.0,Their work showed that each equilibrium config...,Fig. 1. A dual-arm robot manipulating an elast...,1,0
19819,40800,54.0,"In the Kirchhoff model, the wire is allowed to...","Using t \in [0,L] to denote arc-length along t...",1,0


In [82]:
differences.iloc[1]['context_text']
# differences.iloc[1]['text']

'Besides, the routing transitions between different data collection trees contain a nonnegligible time delay and may lead to discontinuity or even loss of the data delivered to the mobile user, which significantly decreases the QoS of ubiquitous data collection.There have been attempts made to efficiently deliver data to mobile users in wireless sensor networks.Most existing works, however, assume that the mobile user has a planned mobility path or the path can be accurately predicted, such that a variety of schemes can be applied to compensate the time cost of the data collection transitions [16], [17].None of those works focus on essentially optimizing the routing transitions, with reduced transition overhead, continuous data delivery, and facilitated data collection for mobile users with unlimited mobility paths.'